In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C, Datasets as D, Parsers as P, Utilities as U
from mppi.Utilities import Tools
from mppi.Datasets import PostProcessing as PP
from mppi.Utilities import Constants as Const
import matplotlib.pyplot as plt
import numpy as np
import os, yaml
#import pybinding as pb

In [4]:
# RunRules for local computations
mpi = 4
omp = 2

rr = C.RunRules(omp_num_threads=omp,mpi=mpi)
code = C.QeCalculator(rr)
code.global_options()

Initialize a QuantumESPRESSO calculator with scheduler direct


{'scheduler': 'direct',
 'mpi': 4,
 'omp_num_threads': 2,
 'executable': 'pw.x',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': False,
 'verbose': True}

In [ ]:
# RunRules for ismhpc
# The product of ntasks_per_node*cpus_per_task is equal to 32. 
# Many mpi are needed for better performances
nodes = 1
ntasks_per_node = 16
cpus_per_task=2
omp_num_threads=2

ntasks = nodes*ntasks_per_node

rr = C.RunRules(scheduler='slurm',partition='all12h',
                memory='125000',time='11:59:00',
                nodes=nodes,ntasks_per_node=ntasks_per_node,
                cpus_per_task=cpus_per_task,
                omp_num_threads=omp_num_threads)
code = C.QeCalculator(rr,activate_BeeOND=True) #,skip=False,clean_restart=False
#code.global_options()

In [5]:
# RunRules for m100
nodes = 1
ntasks_per_node = 4
cpus_per_task=32
omp_num_threads=8
gpus_per_node=4
pe=8

ntasks = nodes*ntasks_per_node

rr = C.RunRules(scheduler='slurm',partition='m100_usr_prod',
                account='IscrC_AlNWS2',
                memory='120GB',time='23:59:00',
                nodes=nodes,ntasks_per_node=ntasks_per_node,
                cpus_per_task=cpus_per_task,omp_num_threads=omp_num_threads,
                gpus_per_node=gpus_per_node,pe=pe,
                map_by='socket',rank_by='core')
code = C.QeCalculator(rr)
#code.global_options()

# DFT analysis of the 2D MoS$_2$

This notebook contains the DFT analysis of the 2D MoS$_2$ performed with QuantumESPRESSO

## Lattice properties and atomic positions

We use the lattice configuration adopted by QuantumESPRESSO in the ibrav=4 case.

The basis vector of the (direct) lattice are defined as
$$
a_1 = a_{lat}(1,0,0) \, ,\quad a_2 = a_{lat}(\frac{-1}{2},\frac{\sqrt{3}}{2},0) \, ,\quad
a_3 = a_{lat}(0,0,\frac{c}{a_{lat}})
$$
so that $c$ vacuum space between two replica of the sheets, expressed in the same units as $a_{lat}$.
We also observe that $\textrm{celldm(3)}=c/a_{lat}$, so it expresses the vacuum distance in terms of 
$a_{lat}$.

In cartesian coordinates the atomic positions read
$$
Mo = a_{lat}(0,\frac{1}{\sqrt{3}},0) \, \qquad S = a_{lat}(\frac{1}{2},\frac{1}{2\sqrt{3}},\delta) \, \quad
S = a_{lat}(\frac{1}{2},\frac{1}{2\sqrt{3}},-\delta)
$$
where $\delta$ is the buckling of the $S$ atoms in units of $a_{lat}$. Converted in crystal coordinates the atomic positions read
$$
Mo = (\frac{1}{3},\frac{2}{3},0) \, \qquad S = (\frac{2}{3},\frac{1}{3},\delta_{cryst}) \, \quad
S = (\frac{2}{3},\frac{1}{1},-\delta_{cryst})
$$
where $\delta = (c/a_{lat})\delta_{cryst}$, so the cartesian buckling is equal to the crystal one multiplied times celldm(3).

In [48]:
0.073413*c

0.49770840541744893

In [50]:
c*alat

39.99999999949244

## GS Analysis

We compute the electronic configuration of the GS.

In [16]:
pseudo_dir = '../pseudos'
Mo_pseudo = 'Mo-fr.out'
S_pseudo = 'S-fr.out'

According to the results provided by Fulvio Paleari the converged atomic positions read 

In [51]:
alat = 5.9000811881 # in a.u. (Bohr)
c = 40.0 # vacuum distance in a.u. 
celldm3 = c/alat # in units of alat
delta_cryst = 0.073413577 # buckling in crystal coordinates

kpoints = [12,12,1]
ecut_wf = 140.0 # energy cutoff of the wfs in Ry

Atomic positions in crystal coordinates

In [18]:
Mo = np.array([1./3.,2./3.,0.])
S1 = np.array([2./3.,1./3.,delta_cryst])
S2 = np.array([2./3.,1./3.,-delta_cryst])

We build the input

In [34]:
prefix = 'gs_analysis'

inp = I.PwInput()
inp.set_scf(force_symmorphic=True,conv_thr=1e-6)
inp.set_prefix(prefix)
#inp.set_num_bnds(32)
inp.set_pseudo_dir(pseudo_dir=pseudo_dir)
inp.set_spinorbit()
inp.add_atom(atom='Mo',pseudo_name=Mo_pseudo,mass=42.00)
inp.add_atom(atom='S',pseudo_name=S_pseudo,mass=16.00)
inp.set_atoms_number(3)
inp.set_atomic_positions([['Mo',Mo],['S',S1],['S',S2]],type='crystal')
inp.set_lattice(ibrav=4,celldm1=alat,celldm3=celldm3)
inp.set_kpoints(type='automatic',points=kpoints)
inp.set_energy_cutoff(ecut_wf)
inp

{'control': {'calculation': "'scf'",
  'verbosity': "'high'",
  'prefix': "'gs_analysis'",
  'outdir': "'./'",
  'pseudo_dir': "'../pseudos'"},
 'system': {'force_symmorphic': '.true.',
  'lspinorb': '.true.',
  'noncolin': '.true.',
  'ntyp': '2',
  'nat': '3',
  'ibrav': 4,
  'celldm(1)': 5.9000811881,
  'celldm(3)': 6.7795677253,
  'ecutwfc': 140.0},
 'electrons': {'diago_full_acc': '.false.', 'conv_thr': 1e-06},
 'ions': {},
 'cell': {},
 'atomic_species': {'Mo': [42.0, 'Mo-fr.out'], 'S': [16.0, 'S-fr.out']},
 'atomic_positions': {'type': 'crystal',
  'values': [['Mo', array([0.33333333, 0.66666667, 0.        ])],
   ['S', array([0.66666667, 0.33333333, 0.07341358])],
   ['S', array([ 0.66666667,  0.33333333, -0.07341358])]]},
 'kpoints': {'type': 'automatic', 'values': ([12, 12, 1], [0.0, 0.0, 0.0])},
 'cell_parameters': {}}

And we run the computation

In [35]:
run_dir = 'GS'

In [36]:
code.run(input=inp,run_dir=run_dir,name=prefix)

delete log file: GS/gs_analysis.log
run command: mpirun -np 4 pw.x -inp gs_analysis.in > gs_analysis.log
computation gs_analysis is running...


Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL


computation gs_analysis ended


'/home/marco/Data/RICERCA/DFT AND MANY BODY/2D ELECTRONIC AND OPTICAL PROPERTIES/MoS2/GS/gs_analysis.save/data-file-schema.xml'

## Band structure on high symmetry line

In [5]:
######################################################

According to the original input provided by Fulvio Paleari the Mo and the two S atoms are located in 
$$
Mo = \left(\frac{1}{3},\frac{2}{3},0 \right) \\
S =  \left(\frac{2}{3},\frac{1}{3},0.073413577 \right) \\
S = \left(\frac{2}{3},\frac{1}{3},-0.073413577 \right)
$$
expressed in crystal coordinates. 
$$
Mo = a_{lat}\left(0,\frac{1}{\sqrt{3}},0 \right) \\
S =  \left(\frac{2}{3},\frac{1}{3},0.073413577 \right) \\
S = \left(\frac{2}{3},\frac{1}{3},-0.073413577 \right)
$$